## Approach 2 - resample from the smaller class to match the size of the larger class

In [1]:
import os
import pydub
from python_speech_features import mfcc
import librosa
import pandas as pd
import numpy as np
import pickle
import random
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load(name):
    file = open(name,'rb')
    input_ = pickle.load(file)
    file.close()
    
    zone = {}
    for idx,key in enumerate(input_):
        zone[key] = input_[key]
    
    return zone 

In [3]:
def get_mfcc(wav_rate):
    """
    return the mfcc features  
    """
    wav, sample_rate = wav_rate
    features = librosa.feature.mfcc(wav, sr=sample_rate,n_mfcc=40)
    return features 

In [4]:
zone1 = load("class1_wav_1.pkl")
zone1 = pd.DataFrame.from_dict(zone1.items())
zone1.columns = ['id', 'wav_rate']
zone4 = load("class1_wav_4.pkl")
zone4 = pd.DataFrame.from_dict(zone4.items())
zone4.columns = ['id', 'wav_rate']
zone8 = load("class1_wav_8.pkl")
zone8 = pd.DataFrame.from_dict(zone8.items())
zone8.columns = ['id', 'wav_rate']
zone13 = load("class1_wav_13.pkl")
zone13 = pd.DataFrame.from_dict(zone13.items())
zone13.columns = ['id', 'wav_rate']
zone = pd.concat([zone1,zone4,zone8,zone13], axis = 0).reset_index(drop=True)
zone['MFCC'] = zone.wav_rate.apply(lambda x: get_mfcc(x))

In [5]:
mfcc = {}
for index in range(len(zone)):
    each = {}
    for key,value in enumerate(zone['MFCC'].values[index][0]):
        each[key] = value
        
    mfcc[index] = each 
class1 = pd.DataFrame.from_dict(mfcc.values())

In [6]:
##create a dataframe (x 120 columns) including metrics like 
##mean, median, and standard deviation  
class1_mean = class1.applymap(np.mean).add_suffix("_mean") 
class1_median = class1.applymap(np.median).add_suffix("_median")
class1_std = class1.applymap(np.std).add_suffix("_std")
class1 = pd.concat([class1_mean,class1_median,class1_std], axis = 1)
class1['target'] = 0 #1 indicates low quality, 0 indicates high quality

In [7]:
class1

,0_mean,1_mean,2_mean,3_mean,4_mean,5_mean,6_mean,7_mean,8_mean,9_mean,...,31_std,32_std,33_std,34_std,35_std,36_std,37_std,38_std,39_std,target
0,-237.691497,144.042031,-67.400611,-25.404225,-1.817005,-30.356081,-28.237392,-9.491172,-17.158020,-17.546154,...,4.983913,4.649618,4.076968,4.814801,4.067516,4.632821,3.883210,4.410012,3.790600,0
1,-204.559032,178.621742,-97.859422,-43.078744,-6.045942,-53.161616,-41.161034,-4.518120,-28.577066,-32.738219,...,3.995709,4.303491,4.641172,4.483399,4.371296,4.134867,4.537295,4.439309,4.570703,0
2,-278.041075,177.613064,-56.803973,-25.328523,11.292544,-21.945024,-22.536982,-4.195929,-15.965443,-20.694936,...,5.476982,5.458801,4.200628,3.406800,3.760255,6.887309,3.918602,4.461580,3.675628,0
3,-195.030372,182.097671,-98.506538,-36.744125,-2.646097,-42.900202,-41.202295,-18.775287,-25.775849,-24.082239,...,4.434588,4.760999,4.650654,4.607592,5.029147,5.106412,4.703493,5.204493,5.524382,0
4,-208.053982,187.873516,-72.659380,-30.777155,-11.001324,-35.832158,-31.272942,-18.761006,-24.380303,-21.050697,...,5.785005,5.558576,4.240495,4.646488,4.130885,4.561608,4.433932,5.529040,5.094990,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57635,-305.889024,174.225742,-135.807705,-43.200104,-6.062359,-78.586819,-37.388511,-24.207833,-49.482120,-14.515135,...,4.803113,4.167430,4.764696,3.318132,3.381130,5.046366,3.872450,3.197658,4.521523,0
57636,-337.135380,142.169007,-178.623461,-35.676171,-7.840126,-88.559753,-19.053542,-14.833515,-46.232593,2.446478,...,3.584695,4.856886,5.347752,5.524782,4.317941,4.536197,5.817503,5.010929,3.904374,0
57637,-382.307511,151.007613,-174.461070,-39.490574,-5.428831,-90.277218,-21.926923,-11.125226,-52.185104,-2.152840,...,6.746811,8.439027,8.102808,10.144276,10.933805,12.886755,10.571148,11.824262,12.833716,0
57638,-314.433105,180.037110,-154.703605,-47.130286,3.384659,-79.243873,-38.011405,-16.951043,-48.877748,-20.480094,...,4.519505,2.306287,3.236903,4.955336,4.323438,4.059283,4.580778,3.287600,3.495599,0


In [9]:
zone1 = load("class2_wav_1.pkl")
zone1 = pd.DataFrame.from_dict(zone1.items())
zone1.columns = ['id', 'wav_rate']
zone4 = load("class2_wav_4.pkl")
zone4 = pd.DataFrame.from_dict(zone4.items())
zone4.columns = ['id', 'wav_rate']
zone8 = load("class2_wav_8.pkl")
zone8 = pd.DataFrame.from_dict(zone8.items())
zone8.columns = ['id', 'wav_rate']
zone13 = load("class2_wav_13.pkl")
zone13 = pd.DataFrame.from_dict(zone13.items())
zone13.columns = ['id', 'wav_rate']
zone = pd.concat([zone1,zone4,zone8,zone13], axis = 0).reset_index(drop=True)
zone['MFCC'] = zone.wav_rate.apply(lambda x: get_mfcc(x))

In [10]:
mfcc = {}
for index in range(len(zone)):
    each = {}
    for key,value in enumerate(zone['MFCC'].values[index][0]):
        each[key] = value
        
    mfcc[index] = each 

#split the 40 mfcc features into 40 columns in dataframe 
class2 = pd.DataFrame.from_dict(mfcc.values()) 

In [11]:
##create a dataframe (x 120 columns) including metrics like 
##mean, median, and standard deviation  
class2_mean = class2.applymap(np.mean).add_suffix("_mean") 
class2_median = class2.applymap(np.median).add_suffix("_median")
class2_std = class2.applymap(np.std).add_suffix("_std")
class2 = pd.concat([class2_mean,class2_median,class2_std], axis = 1)
class2['target'] = 1 #1 indicates low quality, 0 indicates high quality 

In [15]:
class2 = class2.sample(n=57640,replace=True,random_state=5)

In [16]:
class2

,0_mean,1_mean,2_mean,3_mean,4_mean,5_mean,6_mean,7_mean,8_mean,9_mean,...,31_std,32_std,33_std,34_std,35_std,36_std,37_std,38_std,39_std,target
2915,-426.242626,159.549788,-175.141085,-56.544001,9.981021,-83.571585,-41.274238,-1.796990,-50.783374,-16.594937,...,5.499847,5.981191,6.307866,6.768684,7.315853,7.428500,9.629768,11.005077,8.968983,1
2254,-989.125674,-11.935253,-19.973417,20.650758,-2.446034,-5.408903,1.280725,-3.666341,5.083870,-4.342981,...,3.632138,4.590223,3.666906,3.594595,4.127074,3.349240,3.909668,3.422602,3.209814,1
4079,-1003.793735,-17.890375,-21.115031,22.964513,-0.404478,-7.127020,1.355893,-1.322110,5.763811,-8.356008,...,2.712215,4.133809,3.020583,1.732035,0.727157,2.758916,3.165023,2.302534,2.064187,1
1725,-218.424913,197.080772,-71.768556,-34.124177,-22.944706,-35.053344,-21.081571,-9.168166,-8.457799,-15.838909,...,4.641975,5.108509,2.639504,3.875157,3.607050,4.457675,3.685340,4.122555,3.578862,1
3046,-574.312765,56.653229,-45.508936,-23.803654,-0.718349,-18.646847,-13.767262,-6.497310,-12.398608,-5.826463,...,4.260762,2.565937,2.378826,5.224346,2.234862,3.412115,3.619856,3.193501,3.937071,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,-335.644416,150.577981,-83.778798,-61.883728,-14.946121,-57.566028,-46.071827,-15.990635,-34.798993,-25.496329,...,3.982273,3.747169,3.355538,4.049548,3.214920,3.707888,3.911668,3.050625,3.410092,1
6429,-319.330124,152.672306,-82.084638,22.101268,-28.885277,-3.560687,13.611769,-35.906126,5.571268,6.330416,...,7.611991,6.468751,8.138647,7.262803,10.526000,7.196029,8.762047,9.834135,5.846499,1
35,-730.878140,158.362268,-31.504951,18.792081,5.639116,12.256919,16.717029,-0.397628,10.162573,-5.159956,...,4.598363,2.996983,4.062460,4.750476,3.294020,4.016685,3.353429,4.141408,4.118882,1
7718,-393.061813,137.972448,-113.828584,33.734610,-15.467413,-21.805351,24.869058,-13.778591,21.386864,19.948578,...,10.346405,9.041948,9.612609,10.508303,9.647726,8.252176,9.450176,10.909591,9.857962,1


In [17]:
data = pd.concat([class1,class2], axis = 0)
data

,0_mean,1_mean,2_mean,3_mean,4_mean,5_mean,6_mean,7_mean,8_mean,9_mean,...,31_std,32_std,33_std,34_std,35_std,36_std,37_std,38_std,39_std,target
0,-237.691497,144.042031,-67.400611,-25.404225,-1.817005,-30.356081,-28.237392,-9.491172,-17.158020,-17.546154,...,4.983913,4.649618,4.076968,4.814801,4.067516,4.632821,3.883210,4.410012,3.790600,0
1,-204.559032,178.621742,-97.859422,-43.078744,-6.045942,-53.161616,-41.161034,-4.518120,-28.577066,-32.738219,...,3.995709,4.303491,4.641172,4.483399,4.371296,4.134867,4.537295,4.439309,4.570703,0
2,-278.041075,177.613064,-56.803973,-25.328523,11.292544,-21.945024,-22.536982,-4.195929,-15.965443,-20.694936,...,5.476982,5.458801,4.200628,3.406800,3.760255,6.887309,3.918602,4.461580,3.675628,0
3,-195.030372,182.097671,-98.506538,-36.744125,-2.646097,-42.900202,-41.202295,-18.775287,-25.775849,-24.082239,...,4.434588,4.760999,4.650654,4.607592,5.029147,5.106412,4.703493,5.204493,5.524382,0
4,-208.053982,187.873516,-72.659380,-30.777155,-11.001324,-35.832158,-31.272942,-18.761006,-24.380303,-21.050697,...,5.785005,5.558576,4.240495,4.646488,4.130885,4.561608,4.433932,5.529040,5.094990,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,-335.644416,150.577981,-83.778798,-61.883728,-14.946121,-57.566028,-46.071827,-15.990635,-34.798993,-25.496329,...,3.982273,3.747169,3.355538,4.049548,3.214920,3.707888,3.911668,3.050625,3.410092,1
6429,-319.330124,152.672306,-82.084638,22.101268,-28.885277,-3.560687,13.611769,-35.906126,5.571268,6.330416,...,7.611991,6.468751,8.138647,7.262803,10.526000,7.196029,8.762047,9.834135,5.846499,1
35,-730.878140,158.362268,-31.504951,18.792081,5.639116,12.256919,16.717029,-0.397628,10.162573,-5.159956,...,4.598363,2.996983,4.062460,4.750476,3.294020,4.016685,3.353429,4.141408,4.118882,1
7718,-393.061813,137.972448,-113.828584,33.734610,-15.467413,-21.805351,24.869058,-13.778591,21.386864,19.948578,...,10.346405,9.041948,9.612609,10.508303,9.647726,8.252176,9.450176,10.909591,9.857962,1


In [23]:
data.target.value_counts()

0    57640
1    57640
Name: target, dtype: int64

# Model Training 

In [27]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [34]:
scaler = StandardScaler()

In [29]:
y = data.loc[:, data.columns == 'target']

### Mean Features Alone

In [43]:
svc = SVC(kernel = "linear")

In [30]:
X_mean = scaler.fit_transform(data.iloc[:, :40])
X_train, X_test, y_train, y_test = train_test_split(X_mean,y,
                                                    test_size=0.2,
                                                    random_state=2022)
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [32]:
svc.score(X_test, y_test)

0.6196217904233171

In [33]:
parameters = {'C':[0.1, 1, 10]}

In [35]:
X_mean = scaler.fit_transform(data.iloc[:, :40])
X_train, X_test, y_train, y_test = train_test_split(X_mean,y,
                                                    test_size=0.2,
                                                    random_state=2022)
search = GridSearchCV(svc, parameters, cv=10)

In [36]:
search.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 10]})

In [37]:
search.best_params_

{'C': 0.1}

In [38]:
search.score(X_test, y_test)

0.6198386537126995